<a href="https://colab.research.google.com/github/sebasmos/NASA-SpaceApps_Pneuma/blob/master/COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML libraries
import lightgbm as lgb
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import keras
import matplotlib.pyplot as plt

In [0]:
#cargamos el testing set
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Extraemos los datos
#TestingSet = pd.read_csv('drive/My Drive/Colab Notebooks/test.csv')

TrainingSet = pd.read_csv('drive/My Drive/Colab Notebooks/train.csv')
TrainingSet[TrainingSet['Country/Region']=='Colombia']

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
5040,7441,NaN,Colombia,4.5709,-74.2973,2020-01-22,0,0
5041,7442,NaN,Colombia,4.5709,-74.2973,2020-01-23,0,0
5042,7443,NaN,Colombia,4.5709,-74.2973,2020-01-24,0,0
5043,7444,NaN,Colombia,4.5709,-74.2973,2020-01-25,0,0
5044,7445,NaN,Colombia,4.5709,-74.2973,2020-01-26,0,0
...,...,...,...,...,...,...,...,...
5100,7501,NaN,Colombia,4.5709,-74.2973,2020-03-22,231,2
5101,7502,NaN,Colombia,4.5709,-74.2973,2020-03-23,277,3
5102,7503,NaN,Colombia,4.5709,-74.2973,2020-03-24,378,3
5103,7504,NaN,Colombia,4.5709,-74.2973,2020-03-25,470,4


In [0]:
#Extraemos las features TrainingSet
Train_Id            = TrainingSet[['Id']].values
Train_PState        = TrainingSet[['Province/State']].values
Train_CountryRegion = TrainingSet[['Country/Region']].values
Train_Lat     = TrainingSet[['Lat']].values
Train_Long = TrainingSet[['Long']].values
Train_Date = TrainingSet[['Date']].values
Train_Confirmed = TrainingSet[['ConfirmedCases']].values
Train_Fatal = TrainingSet[['Fatalities']].values
# Dictionary 
data = {
    'Country':  Train_CountryRegion,
    'Date': Train_Date,
    'Confirmed' : Train_Confirmed
}

#Extraemos las features TestingSet
'''
Testing_Id = TrainingSet[['Id']].values
Testing_PState = TrainingSet[['Province/State']].values
Testing_CountryRegion = TrainingSet[['Country/Region']].values
Testing_Lat = TrainingSet[['Lat']].values
Testing_Long = TrainingSet[['Long']].values
Testing_Date = TrainingSet[['Date']].values
Testing_Confirmed = TrainingSet[['ConfirmedCases']].values
Testing_Fatal = TrainingSet[['Fatalities']].values
''' 

[['Afghanistan']
 ['Afghanistan']
 ['Afghanistan']
 ...
 ['Zambia']
 ['Zambia']
 ['Zambia']]


"\nTesting_Id = TrainingSet[['Id']].values\nTesting_PState = TrainingSet[['Province/State']].values\nTesting_CountryRegion = TrainingSet[['Country/Region']].values\nTesting_Lat = TrainingSet[['Lat']].values\nTesting_Long = TrainingSet[['Long']].values\nTesting_Date = TrainingSet[['Date']].values\nTesting_Confirmed = TrainingSet[['ConfirmedCases']].values\nTesting_Fatal = TrainingSet[['Fatalities']].values\n"

In [0]:
print("Number of Country/Region: ", TrainingSet['Country/Region'].nunique())
print("Dates go from day",  min(TrainingSet['Date']),"to day", max(TrainingSet['Date']), ", a total of", TrainingSet['Date'].nunique(), "days")
print("Countries with Province/State informed: ", TrainingSet[TrainingSet['Province/State'].isna()==False]['Country/Region'].unique())


Number of Country/Region:  163
Dates go from day 2020-01-22 to day 2020-03-26 , a total of 65 days
Countries with Province/State informed:  ['Australia' 'Canada' 'China' 'Cruise Ship' 'Denmark' 'France'
 'Netherlands' 'US' 'United Kingdom']


In [0]:
# Sequential es el modelo mas simple
model = keras.Sequential()
model.add(keras.layers.Dense(1,input_shape=(1,)))
# indicamos que tan rapido vamos a ajustar el modelo
model.compile(keras.optimizers.Adam(lr=1), 'mean_squared_error')
#entrenar el modelo 30 veces, recorrerla 30 veces. 10 puntos de 
#datos que entran a la red al tiempo
model.fit(Train_Confirmed, Train_Id,epochs = 30, batch_size=10)

In [0]:
#confirmed_total_date = TrainingSet.groupby(['Date']).agg({'ConfirmedCases':['sum']})
#fatalities_total_date = TrainingSet.groupby(['Date']).agg({'Fatalities':['sum']})
#total_date = confirmed_total_date.join(fatalities_total_date)
confirmed_total_date_Colombia = TrainingSet[TrainingSet['Country/Region']=='Colombia'].groupby(['Date']).agg({'ConfirmedCases':['sum']})
print("Number of Cases in Colombia: ", max(confirmed_total_date_Colombia))
fatalities_total_date_Colombia = TrainingSet[TrainingSet['Country/Region']=='Colombia'].groupby(['Date']).agg({'Fatalities':['sum']})
total_date_Colombia = confirmed_total_date_Colombia.join(fatalities_total_date_Colombia)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30,5))
total_date_Colombia.plot(ax=ax1)
ax1.set_title("Colombia confirmed cases", size=14)
ax1.set_ylabel("Number of cases", size=14)
ax1.set_xlabel("Date", size=14)
fatalities_total_date_Colombia.plot(ax=ax2, color='orange')
ax2.set_title("Colombia deceased cases", size=14)
ax2.set_ylabel("Number of cases", size=14)
ax2.set_xlabel("Date", size=14)

Number of Cases in Colombia:  ('ConfirmedCases', 'sum')
           ConfirmedCases
                      sum
Date                     
2020-01-22              0
2020-01-23              0
2020-01-24              0
2020-01-25              0
2020-01-26              0
...                   ...
2020-03-22            231
2020-03-23            277
2020-03-24            378
2020-03-25            470
2020-03-26            491

[65 rows x 1 columns]


In [0]:
#TrainingSet.plot(kind='scatter',x='Train_Id',y='Train_Confirmed',title='Infectados en Colombia')
#y_predicted = model.predict(Train_Confirmed)
#plt.plot(Train_Confirmed,y_predicted, color='red')
total_date_Colombia.describe()

,ConfirmedCases,Fatalities
,sum,sum
count,65.000000,65.000000
mean,39.676923,0.246154
std,105.741889,0.884427
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,3.000000,0.000000
max,491.000000,4.000000


In [0]:
#Confirmed_Patients = confirmed_total_date_Colombia[['ConfirmedCases']].values
#print(Confirmed_Patients)
#Confirmed_Date = confirmed_total_date_Colombia[['Date']].values

dbCopy = TrainingSet.copy

Train_Date = list(TrainingSet[['Date']].values)
Train_Confirmed = TrainingSet[['ConfirmedCases']].values
Dates = []
Confirmed_Patients=[]
for x in TrainingSet[['Date']].values:
  Dates.append(list(x)[0])
#print(Train_Confirmed)
for y in Train_Confirmed:
  Confirmed_Patients.append(list(y)[0])

data = {
    '_Dates':Dates,
    'Confirmed_Patients':Confirmed_Patients
}
print(data)




{'_Dates': ['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01